# __Kaggle movies data analysis and cleaning__

#### Initial setup

In [ ]:
%load_ext watermark
%load_ext autoreload

In [ ]:
%autoreload 2
%watermark -v -n -m -p numpy,pandas,sklego

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

In [ ]:
import pandas as pd
import numpy as np
import sys
import os
from pandas import option_context
from sklego.pandas_utils import log_step
from collections import Counter
sys.path.append('../../../../')
from src.settings import DATA_DIR
CURRENT_PATH = os.path.abspath(os.path.join(os.pardir))
print(CURRENT_PATH)
print(DATA_DIR)

Import functions

In [ ]:
from src.data_processing.dataframe_utils import (drop_unnecessary_cols, rename_cols, reset_index, sort_values, 
                                                start_pipeline, convert_to_numeric, remove_nan, 
                                                remove_duplicates, rearrange_cols, insert_nan)
from src.data_processing.preprocess.kaggle_data_clean import (clean_credits_data, clean_movie_keywords, prepare_cast_crew_table,
                                                            clean_movie_metadata, convert_to_separated_string, merge_movies_metadata_keywords,
                                                            prepare_keywords_table, fix_imdbid, extract_values_to_string)

#### Load dataset

Kaggle films data

In [ ]:
movies_metadata = pd.read_csv(os.path.join(DATA_DIR, 'raw', 'kaggle-the-movies', 'movies_metadata.csv'), low_memory=False)
credits = pd.read_csv(os.path.join(DATA_DIR, 'raw', 'kaggle-the-movies', 'credits.csv'))
movie_keywords = pd.read_csv(os.path.join(DATA_DIR, 'raw', 'kaggle-the-movies', 'keywords.csv'))
links_kaggle = pd.read_csv(os.path.join(DATA_DIR, 'raw', 'kaggle-the-movies', 'links.csv'))

### __Kaggle data__

In [ ]:
def show_uniq_vals(df):
    for col in df:
        print(f'{col} - unique values: {np.unique(df[col].dropna().values).shape}, NaN values: {df[col].isna().any()}')

#### __Movie metadata__

Delete prefix 'tt' from IMDB ids in metadata

In [ ]:
with option_context('display.max_column', None):
    display(movies_metadata.head(2))

In [ ]:
movies_metadata.info()

Drop unnecessary columns and delete 'tt' from 'imdb_id' string value

In [ ]:
drop_cols = ['video', 'poster_path', 'belongs_to_collection', 'popularity', 'homepage',
             'vote_count', 'vote_average', 'production_companies', 'production_countries']

movie_metadata_cleaned = (movies_metadata
                          .pipe(start_pipeline)
                          .pipe(drop_unnecessary_cols, drop_cols)
                          .pipe(fix_imdbid)
                          .pipe(convert_to_separated_string, colname='genres'))

In [ ]:
movie_metadata_cleaned.head(2)

In [ ]:
movie_metadata_cleaned.info()

In [ ]:
movie_metadata_cleaned[movie_metadata_cleaned.overview.isin(['No overview found.', 'No movie overview available.'])].head(2)

In [ ]:
movie_metadata_cleaned = (movie_metadata_cleaned
                          .pipe(insert_nan, ['No overview found.', 'No movie overview available.']))

In [ ]:
movie_metadata_cleaned[movie_metadata_cleaned.overview.isin(['No overview found.', 'No movie overview available.'])].head()

Cast ids to numeric type and clean NaN

In [ ]:
cols_to_numeric = ['id', 'imdb_id']

movie_metadata_cleaned = (movie_metadata_cleaned
                          .pipe(convert_to_numeric, cols_to_numeric)
                          .pipe(remove_nan, cols_to_numeric))

In [ ]:
movie_metadata_cleaned.info()

In [ ]:
movie_metadata_cleaned[movie_metadata_cleaned['imdb_id'].isnull()]

In [ ]:
movie_metadata_cleaned[movie_metadata_cleaned['id'].isnull()]

Drop rows with __duplicated 'imdb_id'__ value

In [ ]:
print('Amount of films by IMDB id in kaggle metadata info: {}'.format(movie_metadata_cleaned.imdb_id.unique().size))

In [ ]:
imdb_counter = Counter(movie_metadata_cleaned.imdb_id.values)
films_imdb_repeated = [(val, count) for (val, count) in imdb_counter.most_common() if count > 1]
print(films_imdb_repeated)
print("------------------------------------")
print('Repeated films by IMDB id amount: {}'.format(len(films_imdb_repeated)))

Repeated films examples

In [ ]:
repeated_imdb_metadata = movie_metadata_cleaned[movie_metadata_cleaned.duplicated(subset=['imdb_id'], keep=False)].copy(deep=True)
repeated_imdb_metadata.sort_values(by=['imdb_id'], inplace=True)

with option_context('display.max_column', None):
    display(repeated_imdb_metadata.head(4))

In [ ]:
drop_nan_cols = ['id', 'imdb_id']
rename_dict = {"id": "kaggle_id"}
sort_cols = ['kaggle_id']

movie_metadata_cleaned = (movie_metadata_cleaned
                          .pipe(remove_duplicates, drop_nan_cols)
                          .pipe(rename_cols, rename_dict)
                          .pipe(sort_values, sort_cols)
                          .pipe(reset_index))

In [ ]:
print('Amount of films by IMDB id after data cleaning: {}'.format(len(movie_metadata_cleaned)))

In [ ]:
movie_metadata_cleaned[movie_metadata_cleaned.duplicated(subset=['imdb_id', 'kaggle_id'], keep=False)].copy(deep=True)

In [ ]:
movie_metadata_cleaned.head(2)

In [ ]:
movie_metadata_cleaned.info()
movie_metadata_cleaned.describe()

In [ ]:
show_uniq_vals(movie_metadata_cleaned)

#### __Credits__

In [ ]:
with option_context('display.max_colwidth', 400):
    display(credits.head(2))

In [ ]:
credits.info()

Film ids are repeated in credits.csv

In [ ]:
credits_counter = Counter(credits.id.values)
credits_ids_repeated = [(val, count) for (val, count) in credits_counter.most_common() if count > 1]
print(credits_ids_repeated)

In [ ]:
repeated_credit_rows = credits[credits.duplicated(subset=['id'], keep=False)].copy(deep=True)
repeated_credit_rows.sort_values(by="id", inplace=True)

with option_context('display.max_colwidth', 400):
    display(repeated_credit_rows.head(4))

Clean credits data from duplicates

In [ ]:
duplicated_cols = ['id']
rename_dict = {"id": "kaggle_id"}
order_cols = ['kaggle_id', 'cast', 'crew']

credits_cleaned = (credits
                   .pipe(start_pipeline)
                   .pipe(clean_credits_data, duplicated_cols, rename_dict, sort_cols,order_cols))

In [ ]:
credits_cleaned.info()

In [ ]:
show_uniq_vals(credits_cleaned)

In [ ]:
credits_cleaned[credits_cleaned.duplicated(subset=['kaggle_id'], keep=False)].head()

In [ ]:
credits_cleaned.isnull().values.any()

#### __Keywords__

In [ ]:
with option_context('display.max_colwidth', 400):
    display(movie_keywords.head())

In [ ]:
movie_keywords.info()

In [ ]:
movie_keywords[movie_keywords.duplicated(subset=['id'], keep=False)].head()

Clean keywords data from blank tags list ('[]') and duplicated movies by id

In [ ]:
drop_rows_idx = movie_keywords.index[movie_keywords.keywords == '[]']
duplicated_cols = ['id']
rename_dict = {"id": "kaggle_id"}
sort_cols = ["kaggle_id"]

movie_keywords_cleaned = (movie_keywords
                          .pipe(start_pipeline)
                          .pipe(clean_movie_keywords, drop_rows_idx, duplicated_cols, rename_dict,
                                sort_cols))

In [ ]:
movie_keywords_cleaned.info()

In [ ]:
show_uniq_vals(movie_keywords_cleaned)

In [ ]:
movie_keywords_cleaned.isnull().values.any()

Extracting plot keywords genome table

In [ ]:
rename_tag_dict = {"id": "keyword_id", "keywords": "plot_keywords"}
sort_tag_cols = ['keyword_id']

plot_keywords_genome = (movie_keywords_cleaned
                        .pipe(start_pipeline)
                        .pipe(prepare_keywords_table)
                        .pipe(rename_cols, rename_tag_dict)
                        .pipe(sort_values, sort_tag_cols)
                        .pipe(reset_index))

In [ ]:
plot_keywords_genome.head()

In [ ]:
plot_keywords_genome.info()

Change format of plot keywords in movie_keywords kaggle table

In [ ]:
movie_keywords_cleaned = (movie_keywords_cleaned
                          .pipe(convert_to_separated_string, colname='keywords'))

In [ ]:
with option_context('display.max_colwidth', 400):
    display(movie_keywords_cleaned.head())

In [ ]:
movie_keywords_cleaned.info()

#### __Links kaggle__

Links kaggle file apply to MovieLens dataset links. 'MovieId' is the same 'movieId' in MovieLens links.csv.

In [ ]:
links_kaggle.head()

In [ ]:
links_kaggle.info()

In [ ]:
show_uniq_vals(links_kaggle)

In [ ]:
print('Amount of unique imdb ids in kaggle links.csv data: {}'.format(len(links_kaggle['imdbId'].unique())))

In [ ]:
links_kaggle[links_kaggle.duplicated(subset=['movieId', 'imdbId'], keep=False)]

In [ ]:
links_kaggle.isnull().values.any()

In [ ]:
drop_cols = ['tmdbId']
links_cleaned = (links_kaggle
                 .pipe(start_pipeline)
                 .pipe(drop_unnecessary_cols, drop_cols)
                 .pipe(reset_index))

In [ ]:
links_cleaned.info()

In [ ]:
links_cleaned[links_cleaned.isna().any(axis=1)].head()

#### __Prepare main movie kaggle data__

Prepare cast-crew movie table

In [ ]:
cast_crew_movies = (credits_cleaned
                    .pipe(start_pipeline)
                    .pipe(prepare_cast_crew_table))

In [ ]:
cast_crew_movies.head()

In [ ]:
cast_crew_movies.info()

In [ ]:
show_uniq_vals(cast_crew_movies)

In [ ]:
cast_crew_movies[cast_crew_movies.duplicated(subset=['kaggle_id'], keep=False)]

Merge cast_crew table with movie metadata table

In [ ]:
rename_dict = {"overview": "storyline", "keywords": "plot_keywords"}
cols_order = ['kaggle_id', 'title', 'original_title', 'genres', 'directors', 'writers', 
              'actors', 'storyline', 'tagline', "plot_keywords"]

main_movie_table = (cast_crew_movies
                    .pipe(start_pipeline)
                    .pipe(merge_movies_metadata_keywords, movie_metadata_cleaned,
                          movie_keywords_cleaned)
                    .pipe(rename_cols, rename_dict)
                    .pipe(rearrange_cols, cols_order)
                    .pipe(reset_index))

In [ ]:
main_movie_table.head(4)

In [ ]:
main_movie_table.info()

In [ ]:
show_uniq_vals(main_movie_table)